<a href="https://colab.research.google.com/github/Leetmus/6S191_MIT_DeepLearning/blob/main/DistilbertwithRandomR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt



my_data = pd.read_csv('persuade_2.0_human_scores_demo_id_github.csv')

my_data.shape
print(my_data.head())

  essay_id_comp                                          full_text  \
0  423A1CA112E2  Phones\n\nModern humans today are always on th...   
1  BC75783F96E3  This essay will explain if drivers should or s...   
2  74C8BC7417DE  Driving while the use of cellular devices\n\nT...   
3  A8445CABFECE  Phones & Driving\n\nDrivers should not be able...   
4  6B4F7A0165B9  Cell Phone Operation While Driving\n\nThe abil...   

   holistic_essay_score  word_count         prompt_name         task  \
0                     3         378  Phones and driving  Independent   
1                     4         432  Phones and driving  Independent   
2                     2         179  Phones and driving  Independent   
3                     3         221  Phones and driving  Independent   
4                     4         334  Phones and driving  Independent   

                                          assignment source_text gender  \
0  Today the majority of humans own and operate c...         NaN      M

In [ ]:
#finding missing values
my_data.isnull().sum()

,0
essay_id_comp,0
full_text,0
holistic_essay_score,0
word_count,0
prompt_name,0
task,0
assignment,0
source_text,13121
gender,0
grade_level,1168


In [ ]:
#filling missing values
from sklearn.impute import KNNImputer

text_cols = my_data.select_dtypes(include=["object", "category"]).columns
for i in text_cols:
    if my_data[i].isnull().sum() > 0:
        if i in ["gender", "ell_status", "race_ethnicity", "prompt_name", "task", "assignment", "economically_disadvantaged", "student_disability_status"]:
            mode_val = my_data[i].mode()[0]
            my_data[i] = my_data[i].fillna(mode_val)
            print(f"Imputed {i} with {mode_val}")
        elif i in ["full_text", "source_text"]:
            my_data[i] = my_data[i].fillna("Missing")
            print(f"Imputed {i} with Missing")
        else:
            mode_val = my_data[i].mode()[0]
            my_data[i] = my_data[i].fillna(mode_val)
            print(f"Imputed {i} with {mode_val}")


# KNN imputation for all numeric columns
numeric_cols = my_data.select_dtypes(include="number").columns
if len(numeric_cols) > 0:
    imputer = KNNImputer(n_neighbors=5)
    my_data[numeric_cols] = imputer.fit_transform(my_data[numeric_cols])
else:
    print("No numeric columns found in my_data")

#  Verifying imputation
print("Missing values after imputation:\n", my_data.isnull().sum())

Imputed source_text with Missing
Imputed ell_status with No
Imputed economically_disadvantaged with Not economically disadvantaged
Imputed student_disability_status with Not identified as having disability
Missing values after imputation:
 essay_id_comp                 0
full_text                     0
holistic_essay_score          0
word_count                    0
prompt_name                   0
task                          0
assignment                    0
source_text                   0
gender                        0
grade_level                   0
ell_status                    0
race_ethnicity                0
economically_disadvantaged    0
student_disability_status     0
dtype: int64


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import torch
import numpy as np
import pandas as pd

# Limit to first 5000 rows (or less for faster testing)
my_data_small = my_data.iloc[:5000].copy()

# Load DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.eval()

# Fill missing full_text
my_data_small["full_text"] = my_data_small["full_text"].fillna("missing")

# Batched DistilBERT embeddings
batch_size = 16
all_cls_embeddings = []

with torch.no_grad():
    for i in range(0, len(my_data_small), batch_size):
        batch_texts = my_data_small["full_text"].iloc[i:i+batch_size].tolist()
        encoded = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        outputs = model(
            input_ids=encoded["input_ids"],
            attention_mask=encoded["attention_mask"]
        )
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token from DistilBERT
        all_cls_embeddings.append(cls_embeddings.cpu().numpy())

# Combine all batches
X_bert = np.concatenate(all_cls_embeddings, axis=0)

# === 3. Other features to include:
other_features = [
    "gender",
    "race_ethnicity",
    "word_count",
    "grade_level",
    "ell_status",
    "economically_disadvantaged",
    "student_disability_status"
]

# === 4. Preprocess structured features ===
cat_cols = my_data_small[other_features].select_dtypes(include=["object", "category"]).columns
num_cols = my_data_small[other_features].select_dtypes(include=["number"]).columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")  # sklearn >=1.2
scaler = StandardScaler()

X_cat = encoder.fit_transform(my_data_small[cat_cols]) if len(cat_cols) > 0 else np.empty((len(my_data_small), 0))
X_num = scaler.fit_transform(my_data_small[num_cols]) if len(num_cols) > 0 else np.empty((len(my_data_small), 0))

# === 5. Combine all features
X_all = np.concatenate([X_bert, X_cat, X_num], axis=1)

# === 6. Define target y
y = my_data_small["holistic_essay_score"].values



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, cohen_kappa_score
import numpy as np

# Initialize and train model
rf = RandomForestRegressor(n_estimators=100, max_depth=4, random_state=42)
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

# Round predictions to nearest class
y_pred_rounded = np.round(y_pred)

# Evaluation
accuracy = np.mean(y_pred_rounded == y_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred_rounded)

print("✅ Accuracy (Rounded):", round(accuracy, 2))
print("📉 MSE:", round(mse, 2))
print("🧠 R^2 Score:", round(r2, 2))
print("🧪 Cohen Kappa Score:", round(kappa, 2))



